In [2]:
# Import library
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# -- Fungsi umum untuk semua permasalahan

# Macam macam algoritma
algorithm_pack = [[GaussianNB(), 'Naive Bayes', 'NB'],
                [KNeighborsClassifier(), 'K Nearest Neighbors', 'KNN'],
                [SVC(), 'Support Vector Machine','SVM'],
                [LogisticRegression(), 'Logistic Regression', 'LR'],
                [GradientBoostingClassifier(random_state=100,
                                            n_estimators=100,
                                            learning_rate=0.07), 'Gradient Boosting', 'GB'],
                [DecisionTreeClassifier(), 'Decision Tree', 'DT'],
                [RandomForestClassifier(), 'Random Forest','RF'],
                [MLPClassifier(max_iter=1000), 'Multi-layer Perceptron', 'MLP']]

# Fungsi untuk menampilkan opsi algoritma apa yang ingin dipakai
def algorithm_option_string():
    string = ''
    for i in range(len(algorithm_pack)):
        string += f"\n{i}. {algorithm_pack[i][1]}"
    string += '\n['
    for i in range(len(algorithm_pack)):
        if(i == len(algorithm_pack) - 1):
            string+= f"{i}]: "
        else:
            string+= f"{i}/"
    return string

# Fungsi untuk print error dengan warna merah dan diberi jarak
def print_error(string):
    print("\n\033[91m"+string+"\033[0m\n")

# Fungsi untuk validasi input opsi integer
def get_integer_option(min_value, max_value, prompt):
    while True:
        try:
            value = int(input(prompt))
            if min_value <= value <= max_value:
                return value
            else:
                print_error(f"Option must be between {min_value} and {max_value}")
        except ValueError:
            print_error(f"Invalid option. Please enter a valid integer value between {min_value} and {max_value}")

# Baca file test sama train
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# -- Variabel umum, dimodifikasi untuk kasus khusus, Tentukan kolom yang ingin di drop dan kolom yang menjadi target
dropped_columns = ['PassengerId', 'HomePlanet', 'Destination', 'Age', 'VIP', 'Name', 'CryoSleep']
target_column = 'Transported'

# Ini untuk memilah fitur agar data mudah diolah kedepannya, fitur yang bersifat kategorikal dan fitur yang bersifat numerikal
categorical_features = train_data.drop(columns=dropped_columns + [target_column]).select_dtypes(include=['object']).columns.tolist()
numerical_features = train_data.drop(columns=dropped_columns + [target_column]).select_dtypes(exclude=['object']).columns.tolist()

# Ini adalah fungsi transformasi yang digunakan untuk fitur kategorikal non ordinal
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),                           # Mengisi null value dengan data modus (yang sering muncul)
    # Jika kode diatas tidak dipakai, berarti menggunakan unkown sebagai fitur
    ('onehot', OneHotEncoder(sparse_output=False).set_output(transform='pandas'))   # Setelah diisi datanya
])                                                                                  # OneHotEncoder mengubah kategorikal menjadi biner (numerikal)

# Ini adalah fungsi transformasi yang digunakan untuk fitur numerikal non ordinal
numerical_imputer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))               # Mengisi null value dengan data median
])

# Untuk menggabungkan kedua transformasi pada 1 step
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_imputer, numerical_features)
    ])

# Agar proses lebih sinkron dan mengurangi kemungkinan terjadinya kesalahan pada sinkronisasi data
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# -- Fungsi khusus permasalahan

# Fungsi buat ngambil huruf S sama P di Side
def extract_last_cabin_part(s):
    if isinstance(s, str):
        parts = s.split('/')
        return parts[-1]
    else:
        return s

# Membersihkan data buat di training

# Drop kolom yang tidak relevan menurut persepsi kelompok terhadap soal (train.csv)
x_train = train_data.drop(columns=dropped_columns + [target_column])
y_train = train_data[target_column]

# -- Fungsi khusus lakukan pembersihan data lanjutan sesuai logika yang dianggap benar (train.csv)
x_train['Cabin'] = x_train['Cabin'].apply(extract_last_cabin_part)

# Mengubah dari data mentah ke data yang akan dilatih (train.csv)
transformed_X_train = pipeline.fit_transform(x_train)

# Drop kolom yang tidak relevan menurut persepsi kelompok terhadap soal (test.csv)
x_test = test_data.drop(columns=dropped_columns)

# -- Fungsi khusus lakukan pembersihan data lanjutan sesuai logika yang dianggap benar (test.csv)
x_test['Cabin'] = x_test['Cabin'].apply(extract_last_cabin_part)

# Mengubah dari data mentah ke data yang akan di uji coba (test.csv)
transformed_X_test = pipeline.transform(x_test)

# -- Fungsi umum permasalahan namun harus disesuaikan dengan output yang diminta oleh soal
# fungsi fungsi berikut hanya untuk memudahkan output

def output_one_csv(i):
    algorithm_pack[i][0].fit(transformed_X_train, y_train)
    predictions = algorithm_pack[i][0].predict(transformed_X_test)
    passenger_ids = test_data['PassengerId']
    output_df = pd.DataFrame({'PassengerId': passenger_ids, 'Transported': predictions})
    output_df.to_csv(f'predictions_{algorithm_pack[i][2]}.csv', index=False)

def output_all_csv():
    for i in range(len(algorithm_pack)):
        output_one_csv(i)

# Chose what do you want to do?
choose = get_integer_option(0, 2, "What do you want to do with the csv?\n0. Train the csv data\n1. Get the info of the csv data\n2. Get the csv of clean preprocessed data\n[0/1/2]: ")

if(choose == 0):
    choose = get_integer_option(0, 1, "Use all model?\n0. No\n1. Yes\n[0/1]: ")
    if(choose == 0):
        choose = get_integer_option(0, len(algorithm_pack) - 1, f"Which model to use?"+algorithm_option_string())
        output_one_csv(choose)
    elif(choose == 1):
        output_all_csv()
elif(choose == 1):
    print("Training data info:\n",train_data.dtypes,"\n")
    print("Test data info:\n",test_data.dtypes)
    print("Categorical Features: ",categorical_features)
    print("Numerical Features: ",numerical_features)
elif(choose == 2):
    # Ambil nama dari kategorikal yang sudah di OneHotEncode
    encoded_categorical_columns = pipeline.named_steps['preprocessor'].transformers_[0][1] \
                                .named_steps['onehot'].get_feature_names_out(categorical_features)

    # menggabungkan nama kolom untuk fitur kategorikal dan fitur numerikal (termasuk 'PassengerId')
    all_column_names = list(encoded_categorical_columns) + numerical_features

    # Memberikan nama kolom ke data yang sudah dibersihkan sebelum diproses
    clean_data_df = pd.DataFrame(transformed_X_train)
    clean_data_df.columns = all_column_names
    clean_data_df.to_csv('clean_data.csv', index=False)


In [2]:
dropped_columns = ['PassengerId', 'HomePlanet', 'Destination', 'Age', 'VIP', 'Name', 'CryoSleep']
target_column = 'Transported'

def extract_last_cabin_part(s):
    if isinstance(s, str):
        parts = s.split('/')
        return parts[-1]
    else:
        return 'U'

x_train['Cabin'] = x_train['Cabin'].apply(extract_last_cabin_part)